In [ ]:
# ADWP.ipynb

import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog
import ipywidgets as widgets
from IPython.display import display
import os

# -------------------------
# Step 1. File selection
# -------------------------
Tk().withdraw()
file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)

if not file_path:
    raise SystemExit("❌ No file selected.")

print(f"Selected file: {file_path}")

# -------------------------
# Step 2. Dropdowns for Year and Month
# -------------------------
current_year = datetime.today().year

year_dd = widgets.Dropdown(
    options=[current_year - 1, current_year, current_year + 1],
    value=current_year,
    description="Year:"
)

month_dd = widgets.Dropdown(
    options=list(calendar.month_name)[1:],  # Jan..Dec
    value=calendar.month_name[datetime.today().month],
    description="Month:"
)

display(year_dd, month_dd)

# -------------------------
# Step 3. Load data
# -------------------------
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")

# Normalize Period Month to first-of-month
df["Period Month"] = pd.to_datetime(df["Period Month"]).dt.to_period("M").dt.to_timestamp()

# -------------------------
# Step 4. Build 24-month horizon
# -------------------------
sel_year = year_dd.value
sel_month = list(calendar.month_name).index(month_dd.value)

start_date = datetime(sel_year, 1, 1)  # Jan of selected year
months = pd.date_range(start=start_date, periods=24, freq="MS")
month_labels = [d.strftime("%b-%y") for d in months]

curr_date = datetime(sel_year, sel_month, 1)
curr_label = curr_date.strftime("%b-%y")

# -------------------------
# Step 5. Processing
# -------------------------
output_records = []

def get_key(row):
    """Return grouping key for PID or MPP case."""
    if pd.notna(row["PID"]):
        return ("PID", row["PID"], row["GCB"], row["Business Framework"])
    else:
        return ("MPP", row["MPP ID"], row["GCB"], row["Business Framework"])

grouped = df.groupby(df.apply(get_key, axis=1))

for key, group in grouped:
    kind = key[0]
    group = group.sort_values("Period Month")

    # Prepare base record
    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # Initialize horizon months
    for label in month_labels:
        record[label] = None

    if kind == "PID":
        # ----- Case 1 & 2: PID present -----
        for _, r in group.iterrows():
            month_label = r["Period Month"].strftime("%b-%y")
            if month_label in record:
                record[month_label] = r["FTE"]

        # Determine latest FTE up to current month
        available_months = [r["Period Month"].strftime("%b-%y") for _, r in group.iterrows()]
        latest_before_curr = None
        for m in month_labels:
            if m in available_months and datetime.strptime(m, "%b-%y") <= curr_date:
                latest_before_curr = m

        if latest_before_curr:
            record["FTE"] = record[latest_before_curr]
        else:
            record["FTE"] = group.iloc[-1]["FTE"]

        # Forward fill only from current month onward with current FTE
        if curr_label in record and record[curr_label] is not None:
            current_fte = record[curr_label]
            start_fill = month_labels.index(curr_label) + 1
            for label in month_labels[start_fill:]:
                record[label] = current_fte

        output_records.append(record)

    else:
        # ----- Case 3: MPP ID (no PID) -----
        event = group.iloc[0]
        event_month = event["Period Month"].strftime("%b-%y")
        fte_val = event["FTE"]

        record["FTE"] = fte_val
        if event_month in month_labels:
            start_idx = month_labels.index(event_month)
            for label in month_labels[start_idx:]:
                record[label] = fte_val

        output_records.append(record)

# -------------------------
# Step 6. Build DataFrame
# -------------------------
out_df = pd.DataFrame(output_records)

# Sorting
sort_cols = [
    "Business Service L2",
    "Business Framework Group",
    "Business Framework",
    "Country R1",
    "Country R2",
    "Country R3",
    "PID",
    "MPP ID",
    "GCB"
]
out_df = out_df.sort_values(sort_cols, na_position="last").reset_index(drop=True)

# -------------------------
# Step 7. Save Output
# -------------------------
out_file = os.path.join(
    os.path.dirname(file_path),
    "ADWP_Output.xlsx"
)
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

print(f"✅ ADWP Output written to: {out_file}")
